<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/RTKlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/Gdrive')

# RTKlib.ipynb 
Online Processing of raw "survey grade" GNSS data using [RTKlib](http://www.rtklib.com/) and [rtkexplorer](http://rtkexplorer.com/) and [UNAVCO's Teqc](https://www.unavco.org/software/data-processing/teqc/teqc.html).

C. W. Wright Consulting. https://github.com/lidar532 

(C) 2017,2018,2019, 2020

Online Store: [Lidar.net](https://lidar.net)

Expand this cell for help and more information.


## How to use RTKlib.ipynb
RTKlib.ipynb runs completly online within [Google Colabs](https://colab.research.google.com/). No need to install RTKlib, Python, or anything else on your computer.  Simply upload your raw receiver data to Colabs, Open and run each cell, and your data will be processed and a precision trajectory will be generated.  Sample data are also provided.  Use the upload button on the left panel (select the file folder icon to show the file area) to upload your GNSS data for processing.  Once your data are processed, download the results from the File pane area.

* **RTKlib Install:** Installs the Linux version of RTKlib on your Google Colabs virtual machine. To install, simply click the: `[ ] rtklib_install(): Install RTKlib`.  To install sample datasets, click the: `[ ] !git clone. Install Sample Datasets`
* **RTKlib Convert raw to RINEX:** Use this cell to convert raw GNSS data to RINEX for use with RTKlib.  First, click:

 `[ ] def rtkconvert(). Convert RAW to RINEX` to load suporting functions, and then click:

 `[ ] GUI: Convert a CWW-PPK raw GPS data file to RINEX for processing.`  Copy and paste a raw file name into the `Raw_File_Name` text input box.  The `Data_source` is preloaded with various UAS; however, you can simply enter another name in the associated text box.  `Marker_Name` and `User_Comment` can be anything you like.  These data will be used to populate the resulting RINEX header section.  After filling your data, simply click the`[ ]` to run the conversion.  The command will rename your input file and all output files based to reflect the date and time the data were collected.

* **Teqc RINEX Quality Check:** Use UNAVCO's Teqc to examine the quality of any RINEX file.
  * `[ ] Teqc +qc. A RINEX Observation (*.Obs or *.19o) file.`  Paste a RINEX file name in the input field, and then press the `[ ]` to examine and generate a Teqc report.  This is an optional step and is not required to process data.


 * **RTKlib. Prep RINEX for upload to OPUS. (Resample to lower rate):** This cell will prepare your base station data to be sent to OPUS.  You can skip this step if you are either using an existing CORS stations, or you know the precise position of your base station.  To use this feature, execute the `[ ] def prep_for_opus()` to load necessary functions, and then populate the fields in the `[ ] GUI: Prep RTKlib Local Base RINEX to send to OPUS or to use with RTKlib (static).` cell.  Once the fields are populated, press the `[ ] Gui:` button.  Download the results, and send to OPUS.

 * **RTKlib functions:** Execure the following cells to load required functions:
    * `[ ] dms2dd( dms )` 
    * `[ ] rtklib_print_settings( settings ); Display the Kinematic Processing Settings.` Only execute this function if you want to examine all of the RTKlib settings that will be used to process your data.

 * **RTKlib. Configure Base and General**
    * `[ ] rtklib_general_config(Base_RINEX).` Run this cell to load required functions.
    * `[ ] (Required data) Load Base Station RINEX Files and settings to use.` Populate the fields with your data.  The `Base_RINEX` field can be populated with one or more comma (",") separated base station observation files.  The system will sequentially process each rover against each base station.  
    There is no need to enter additional nav, gnav, or sp3 files as those contain the same data. The `Base_Longitude_X`, `Base_Latitude_Y`, and `Base_Elevation_Z` values must be in the same format as used within an OPUS Report and they are not required at all if you select `RINEX Header` or `OPUS Report` for `Base_Coordinates_Source`.  
    If you upload an OPUS Report, populate `OPUS_Report_File` with the name, the program can automatically extract the base station coordinates from the file and populate the base position as required by RTKlib.

* **RTKlib. Process.** Generates PPK trajectories.  Populate and execute the following cells to generate trajectories:
  * `[ ] rtklib_process()` Execute to load required functions.
  * `[ ] Generate an RTKlib GNSS Trajectory.` Populate the fields, and select the RTKlib options, and then press the `[ ]` to generate trajectories.  You can put more than one rover/UAS observation file in the `Aircraft_RINEX` field.  The program will process each rover/UAS file against each base station observation file loaded into the `Aircraft_RINEX` field in the `[ ] RTKlib. Configure Base and General` section.

* **Generated output files:**
  * **\*-pos.txt**:  RTKlib position output file. 
  * **\*-elev.kmz**: Kmz version of the **\*-pos.txt** with elevation displayed.
  * **\*-flat.kmz**: Kmz version of the **\*-pos.txt** with flattened elevation displayed.
  * **\*-conf.txt**:  Configuration settings used to generate the **\*-pos.txt** file.



# A. RTKlib Install. (Expand and run A1.0 & A2.0 to get started )

In [ ]:
#@title Step A1.0 Mount your Google Drive as /content/Gdrive 
from google.colab import drive
!rm -rf /content/sample_data/
drive.mount('/content/.gdrive')
!ln -sf /content/.gdrive/My\ Drive/ /content/Gdrive
print('Your Gdrive is mounted')

In [ ]:
#@title Step A2.0 Load functions, defs, and code.     Install RTKlib { form-width: "45%" }
import os
import sys
from IPython.core.display import HTML, display

def rtklib_install():
  """
  Begin configuring for RTKlib and processing GNSS Data.
  """
  print('Download and install RTKlib from the lidar532 Github.')
  !rm -rf /content/sample_data
  rv = ! which rnx2rtkp
  if rv == []:
    !cd /usr/local/bin; rm  -rf convbin rnx2rtkp pos2kml rtkrcv str2str
    !cd /content/;       rm -rf RTKLIB
    !cd /usr/local/src/; rm -rf RTKLIB
    !cd /usr/local/src; git clone https://github.com/lidar532/RTKLIB.git
    !cd /usr/local/src/RTKLIB/app/; make install
    !cd /usr/local/bin; chmod uog+x rnx2rtkp convbin pos2kml rtkrcv str2str
  else:
    print('Skipping Install. RTKlib programs were alread installed.')

  # Get and install teqc
  rv = ! which teqc
  if rv == []:
    print('Download and install Teqc from Unavco.')
    !wget https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip
    !unzip teqc_Lx86_64s.zip
    !mv teqc /usr/local/bin
    !rm -rf teqc_Lx86_64s.zip
  else:
    print('Skipping Install. Teqc alread installed.')

  # Install geprinex package.
  !pip install georinex
  return

rtk_lib_loaded = True

#@title RTKlib & Teqc Install.{form-width: "25%"}
if __name__ == '__main__':
  rtklib_install()
  print('RTKlib binaries installed.')

def github_clone(url, branch, pth, pyfile ):
  fp = f'{pth}/{pyfile}'
  if os.path.exists(fp):
    print(f'{fp} exists.  Skipping Cloning.')
  else:
    print(f'cloning to {fp}')
    ! id;\
      rm -rf {pth};\
      mkdir -p {pth};\
      cd {pth};\
      git clone -v -b {branch}  {url} {pth};\
      jupyter-nbconvert --to python {pyfile} 
    sys.path.append(pth)
    
###############################################################
# Install all the functions from the twin python module.  
# For development or debugging of any functions set
# enable_local_dev = True, otherwise set it to False.
###############################################################
if __name__ == '__main__':
  enable_local_dev =     True
  print(f'Using local functions:{enable_local_dev}')
  if enable_local_dev == False:
    github_clone('https://github.com/lidar532/ppkgeotag', '2020-1020-dev', '/content/rtklib/', 'RTKlib.ipynb' )
    from RTKlib import *
#@title defs. Load the rtklib settings variable k_settings
# setup the options file that will be sent to the RTKlib processor.
k_settings = { 
    'pos1-soltype'      : 'combined     # (0:forward,1:backward,2:combined)',     
    'pos1-posmode'      : 'kinematic    # (0:single,1:dgps,2:kinematic,3:static,4:movingbase,5:fixed,6:ppp-kine,7:ppp-static)',
    'pos1-frequency'    : 'l1+l2        # (1:l1,2:l1+l2,3:l1+l2+l5)',        
    'pos1-elmask'       : 12,             
    'pos1-snrmask'      : 5.0,
    'pos1-dynamics'     : 'off',
    'pos1-tidecorr'     : 'off',
    'pos1-ionoopt'      : 'brdc         # (0:off,1:brdc,2:sbas,3:dual-freq,4:est-stec)',
    'pos1-tropopt'      : 'saas         # (0:off,1:saas,2:sbas,3:est-ztd,4:est-ztdgrad)',
    'pos1-sateph'       : 'precise      # (0:brdc,1:precise,2:brdc+sbas,3:brdc+ssrapc,4:brdc+ssrcom)',
    'pos1-exclsats'     : '',
    'pos1-navsys'       : '5            # (1:gps+2:sbas+4:glo+8:gal+16:qzs+32:comp)',
    'pos2-armode'       : 'continuous   # (0:off,1:continous,2:instantaneous,3:fix-and-hold)',
    'pos2-gloarmode'    : 'on',
    'pos2-arthres'      : 3,
    'pos2-arlockcnt'    : 5,
    'pos2-arelmask'     : 0,
    'pos2-aroutcnt'     :5,
    'pos2-arminfix'     :10,
    'pos2-slipthres'    :0.05,
    'pos2-maxage'       :30,
    'pos2-rejionno'     :30,
    'pos2-niter'        :1,
    'pos2-baselen'      :0,
    'pos2-basesig'      :0,
    'out-solformat'     :'llh           # (0:llh,1:xyz,2:enu,3:nmea)',
    'out-outhead'       :'on',
    'out-outopt'        :'on',
    'out-timesys'       :'utc           # (0:gpst,1:utc,2:jst)',
    'out-timeform'      :'hms           # (0:tow,1:hms)'        ,
    'out-timendec'      :6,
    'out-degform'       :'deg'        ,
    'out-fieldsep'      : '',
    'out-height'        :'ellipsoidal' ,
    'out-geoid'         :'internal'   ,
    'out-solstatic'      :'all          # (0:all,1:single)'        ,
    'out-nmeaintv1'      :0          ,
    'out-nmeaintv2'      :0          ,
    'out-outstat'        :'off'        ,
    'stats-errratio'     :100,
    'stats-errphase'     :0.003      ,
    'stats-errphaseel'   :0.003      ,
    'stats-errphasebl'   :0          ,
    'stats-errdoppler'   :10         ,
    'stats-stdbias'      :30         ,
    'stats-stdiono'      :0.03       ,
    'stats-stdtrop'      :0.3        ,
    'stats-prnaccelh'    :1          ,
    'stats-prnaccelv'    :0.1        ,
    'stats-prnbias'      :0.0001     ,
    'stats-prniono'      :0.001      ,
    'stats-prntrop'      :0.0001     ,
    'stats-clkstab'      :5e-12      ,
    'ant1-postype'       :'llh       # (0:llh,1:xyz,2:single,3:posfile,4:rinexhead,5:rtcm)'      ,
    'ant1-pos1'          :0          ,
    'ant1-pos2'          :0          ,
    'ant1-pos3'          :0          ,
    'ant1-anttype'       :'*',
    'ant1-antdele'       :0          ,
    'ant1-antdeln'       :0          ,
    'ant1-antdelu'       :0          ,
    'ant2-postype'       :'rinexhead          # (0:llh,1:xyz,2:single,3:posfile,4:rinexhead,5:rtcm)',
    'ant2-pos1'          :'35.1320570679997   # lat',
    'ant2-pos2'          :'139.624306577      # Lon',
    'ant2-pos3'          :'73.907699999947    # Elevation' ,
    'ant2-anttype'       :'*',
    'ant2-antdele'       :0          ,
    'ant2-antdeln'       :0          ,
    'ant2-antdelu'       :0          ,
    'misc-timeinterp'    :'on'         ,
    'misc-sbasatsel'     :0          ,
    'file-satantfile'    :'/usr/local/src/RTKLIB/data/igs08.atx',
    'file-rcvantfile'    :'/usr/local/src/RTKLIB/data/igs08.atx',
    'file-staposfile'    :'/usr/local/src/RTKLIB/data//stations.pos',
    'file-geoidfile'     : '',
    'file-dcbfile'       :'/usr/local/src/RTKLIB/data/P1C1_ALL.DCB',
    'file-tempdir'       : '/tmp/',
    'file-geexefile'     : '',
    'file-solstatfile'   : '',
    'file-tracefile'     : ''
    }

#@title def rtkconvert().  Convert RAW to RINEX
import re
import os as os
import georinex as gr
def rtkconvert(fn, src, marker, comment):
  if os.path.exists(fn):
    # Build up command string for the RTKlib convbin program.
    c1='-hc \''+comment+'\' '
    c2='-hc \''+src+'\' '
    hm='-hm \''+marker+'\' '
    fn_root = str.split(fn,"/")[-1]
    fn_root = str.split(fn_root,'.')[0]
    cmd = '/usr/local/bin/convbin -os -od -f 2 -ti 1 -r nov '+c1+hm+fn
    print(cmd)
    print('Converting raw to RINEX')
    os.system( cmd )
    #!head -20 /content/GP212112.obs


    # Extract the base file name after all '/' and before any '.'
    orig_root_fn = os.path.basename(fn)
    orig_root_fn = os.path.splitext(orig_root_fn)[0]
    orig_path  = os.path.dirname(fn)

    # build up path name for the RINEX .obs file that should now exist
    fobs = orig_path+'/'+orig_root_fn+'.obs'

    # extract the RINEXT Header information to get the data & time and other fields
    print('Reading the RINEX obs, extracting data & time.')
    rnx=gr.rinexheader(fobs)
    rnx['t0'].date(), rnx['t0'].time()
    ofn = format(rnx['t0'], '%Y-%m%d-%H%M%S-')+src+'-'+fn_root

    # Rename all of the generate RINEX files using the computed root name.
    d = os.listdir(orig_path)
    print('Renaming all of the RINEX files')
    for i in d:
      if re.match( os.path.splitext(i)[0], orig_root_fn  ):
        ext = os.path.splitext(i)[1]
        if ext == 'TXT':
          print(f'TXT Found: {i}')
        new_fn = orig_path+'/'+ofn+ext
        new_fn = re.sub(r"[\n\t\s]*", "", new_fn)
        old_fn = orig_path+'/'+i
        print(old_fn, new_fn)
        os.rename(old_fn, new_fn)
    print('Operation completed.')
  else:
    print(fn+' not found.')

#@title def dms2dd( dms )  {form-width: "25%"}
# convert the lat/lon strings found in OPUS reports to usable float values
def dms2dd( dms ):
  """
  Convert dms (Degrees Minutes Seconds) strings to double.
  """
  rv = -999
  strt = type('x')
  if type(dms) == strt:
    dms = dms.split()  
  if type(dms) == float or type(dms)  == int:
    rv =  float(dms)
  elif len(dms) == 1:
    rv = float(dms[0])
  elif len(dms) == 5:
    dms[0] = dms[0].upper()
    rv = int(dms[2]) + int(dms[3])/60.0 + float(dms[4])/3600.0
    if ( dms[0] == 'S' or dms[0] == 'W'):
      rv = -rv
    return rv
  elif len(dms) == 4:
      dms[0] = dms[0].upper()
      rv = int(dms[1]) + int(dms[2])/60.0 + float(dms[3])/3600.0
      if ( dms[0] == 'S' or dms[0] == 'W' ):
        rv = -rv
  return rv

junk='''
#@title Test
# Testing code goes below
if __name__ == '__main__':
  print('Test data.')
  s = ['N 38  30 0.0', 
       'S 38  30 0.0',
       'W 105 30 0.0', 
       'E 105 30 0.0'      
  ]
  for v in s:
    print(f"Input: {v}   to: {dms2dd(v):12.7f}")
'''

#@title def. rtklib_print_settings( settings );  Display the Kinematic Processing Settings. {form-width: "25%"}
def rtklib_print_settings( settings ):
  """
  Prints out RTKlib settings held in settings.

  """
  for k in settings:
    print(f'{k:<16}  :{k_settings[k]:<60}')

# Function to extract a single variable from a file.
def get_file_var( fn, var ):
  """
  """
  for n, line in enumerate(open(fn)):
    if var in line:
      return line
 
junk = ''' 
#@title Dump rtklib settings
if __name__ == '__main__':
  rtklib_print_settings( k_settings )
'''


#@title F0.0 rtklib_process()  {form-width: "25%"}
def rtklib_process(rk, Aircraft_RINEX, Base_RINEX):
  """
  """
  eph_types = { 'Precise':    'precise',
                'Rapid' :     'precise',
                'Ultra Rapid':'precise',
                'Broadcast' : 'brdc'
                }
  svs = { 'GPS' : 1, 
          'GPS_SBAS' : 3,
          'GPS+GLONASS' : 5 }


  # Copy user configured settings to the configuration data structure.
  k_settings['pos1-sateph' ] = eph_types[rk['Ephemeris_Type']]
  k_settings['pos1-posmode'] = rk['Position_Mode']
  k_settings['pos1-soltype'] = rk['Solution_Type']
  k_settings['pos1-elmask' ] = rk['Elevaton_Mask']
  k_settings['pos1-navsys' ] = svs[ rk['Satellite_Systems'] ]

  if rk['Base_Coordinates_Source'] == 'RINEX Header':
    k_settings['ant2-postype' ] = 'rinexhead'  
  elif rk['Base_Coordinates_Source'] == 'Lat Lon Elevation': 
    k_settings['ant2-postype' ] = 'llh'
    k_settings['ant2-pos1'    ] = rk['Base_Latitude_Y']
    k_settings['ant2-pos2'    ] = rk['Base_Longitude_X']
    k_settings['ant2-pos3'    ] = rk['Base_Elevation_Z']
  elif rk['Base_Coordinates_Source'] == 'X Y Z':
    k_settings['ant2-postype' ] = 'xyz'
    k_settings['ant2-pos1'    ] = rk['Base_Latitude_Y']
    k_settings['ant2-pos2'    ] = rk['Base_Longitude_X']
    k_settings['ant2-pos3'    ] = rk['Base_Elevation_Z']

  try:
    rtk_lib_loaded
    import re
    import os as os
    import georinex as gr
    import pathlib
  except NameError:
    print('****************************************************************************************')
    print('*    Run the above  cell above to configure tools before using this cell.              *')
    print('****************************************************************************************') 
  else:
    if os.path.exists(Aircraft_RINEX):
      # Build up the output Trajectory file name
      bn = get_file_var(Base_RINEX, 'MARKER NAME').split()[0]
      otfn_mode  =   {'kinematic' : 'K', 'single'   : 'C',        'dgps' : 'D',    'static' : 'S', 'fixed' : 'F', 'ppp-static' : 'PPPS'}
      otfn_etype =   {'Precise'   : 'P', 'Rapid'    : 'R', 'Ultra Rapid' : 'U', 'Broadcast' : 'B' }
      otfn_navsys=   {'GPS'       : 'N', 'GPS_SBAS' : 'W', 'GPS+GLONASS' : 'GN' }
      otfn_direction={'combined'  : 'cmb', 'forward': 'fwd', 'backward'  : 'rev'}

      otfn_root = str.split(
          os.path.basename(Aircraft_RINEX),
          '.')[0] + '-'+\
          bn+'-'+ \
          otfn_etype[Ephemeris_Type] +  \
          otfn_mode[Position_Mode] + \
          str(k_settings['pos1-elmask']) + \
          otfn_navsys[rk['Satellite_Systems']] + '-' + \
          otfn_direction[k_settings['pos1-soltype']]

      if os.path.isdir(rk['Trajectory_Folder']) == False:
        os.makedirs(rk['Trajectory_Folder'])


      # Write the custom configured options out to a configuration file.
      otfn_conf = rk['Trajectory_Folder']+'/'+otfn_root+'-conf.txt'   
      otf_conf = open(otfn_conf, 'w+' )
      for i in k_settings:
        print( i, '\t\t=', k_settings[i], file=otf_conf, sep='' )
      otf_conf.close()
      
      otfn           = rk['Trajectory_Folder']+'/'+otfn_root+'-pos.txt'
      otfn_kml       = rk['Trajectory_Folder']+'/'+otfn_root+'-pos-flat.kml'
      otfn_kmz       = rk['Trajectory_Folder']+'/'+otfn_root+'-pos-flat.kmz'
      otfna_kml      = rk['Trajectory_Folder']+'/'+otfn_root+'-pos-elev.kml'
      otfna_kmz      = rk['Trajectory_Folder']+'/'+otfn_root+'-pos-elev.kmz' 
      otfn_errors    = rk['Trajectory_Folder']+'/'+otfn_root+'-errors.txt'   
      # Build up command string for the RTKlib post processor program.
      # Use the root file name of the Aircraft file.
      cmd = '/usr/local/bin/rnx2rtkp  -t \\\n\
          -k '+otfn_conf+' \\\n\
          -o ' + otfn +          '\\\n\t' \
            + Aircraft_RINEX + " \\\n\t" \
            + Base_RINEX +     " \\\n\t" \
            + rk['Base_RINEX_Nav'] + " \\\n\t" \
            + rk['Base_RINEX_Gnav'] + " \\\n\t" \
            + rk['Base_GPS_SP3'] +   " \\\n\t" \
            + rk['Base_GLO_SP3'] +   " \\\n\t" \
            + f" 2>{otfn_errors} >stdout.txt"
      print(f'Cmd:{cmd}')
      if rk['Generate_Output_Trajectory'] == 'Yes':
        print('**************************************************')
        print('* Processing.  This may take several minutes     *')
        print('**************************************************') 
        os_rv = os.system( cmd )

        # Generate a kml from trajectory file.
        rv = !pos2kml  -tu -q 0 -o     {otfn_kml}  {otfn}
        rv = !pos2kml  -tu -ag -q 0 -o {otfna_kml} {otfn}
        #convert the kml to a kmz
        !zip -q {otfn_kmz} {otfn_kml};    rm {otfn_kml}
        !zip -q {otfna_kmz} {otfna_kml};  rm {otfna_kml}      
        if rv: 
          print(rv)
        print(f'Trajectory generated: {otfn}\
          \nOperation completed.\n\n')
        #!cat errors.txt
        !rm -rf {rk['Trajectory_Folder']+'/*-pos_events.pos'}
      else:
        print('No output file generated.\n\n')
        #print('Command: ', cmd)
    else:
      print(Aircraft_RINEX+' not found.')


def mprocess(rk):
  for f in rk['Aircraft_RINEX'].split(','):
    f = f.strip()
    for bfn in rk['Base_RINEX'].split(','):
      bfn = bfn.strip()
      print(f'Processing Aircraft: {f}')
      print(f'         Using base: {bfn}')
      rv = rtklib_process(rk, f, bfn)
      #print(f'rv={rv}
  print('Operations completed.')


  

#@title E0.0 rtklib_general_config(Base_RINEX).  {form-width: "25%"}
def rtklib_general_config(rk):
  """
  Returns: Base_data_ready_for_use
  """
  rk['Base_data_ready_for_use'] = True
  rk['Base_Position_Type']  = 'Not Set'
  print(f"\n\n{rk['Base_RINEX']}")
  # base_files_exist will ultimately remain true only if all of the required files
  # exist.  It does not check for validity (yet).
  rk['base_files_exist'] = True;
  if  os.path.exists(rk['Base_RINEX']) == False:
    print('Base_RINEX file not found: ', rk['Base_RINEX'])
    rk['Base_data_ready_for_use'] = False
    rk['base_files_exist'] = False

  if  os.path.exists(rk['Base_RINEX_Nav']) == False:
    print('Base_RINEX_Nav file not found: ', rk['Base_RINEX_Nav'])
    rk['base_files_exist'] = False
    rk['Base_data_ready_for_use'] = False

  if  (rk['Ephemeris_Type'] != 'Broadcast')  and (os.path.exists(rk['Base_GPS_SP3'])) == False:
    print('Base_GPS_SP3 file not found: ', rk['Base_GPS_SP3'] )
    rk['base_files_exist'] = False
    rk['Base_data_ready_for_use'] = False

  if  (rk['Satellite_Systems']=='GPS+GLONASS'):
    if os.path.exists(rk['Base_RINEX_Gnav']) == False:
      rk['Base_data_ready_for_use'] = False
      print('Gnav file not found: ', rk['Base_RINEX_Gnav'])
      rk['base_files_exist'] = False
      rk['Base_data_ready_for_use'] = False
    if os.path.exists(rk['Base_GLO_SP3']) == False:
      print('Gnav SP3 file not found: ', rk['Base_GLO_SP3'])
      rk['base_files_exist'] = False
      rk['Base_data_ready_for_use'] = False

  if rk['Base_Coordinates_Source'] == 'RINEX Header':
    rk['Base_Position_Type'] = 'rinexhead'
  elif rk['Base_Coordinates_Source'] == 'X Y Z':
    rk['Base_Position_Type'] = 'xyz'
  elif rk['Base_Coordinates_Source'] == 'Lat Lon Elevation':
    rk['Base_Position_Type'] = 'llh'



  if rk['Base_Coordinates_Source']=='OPUS Report':    # if an OPUS file is found, then get coords from it
    if os.path.exists(rk['OPUS_Report_File']):
      opus_lat = get_file_var( rk['OPUS_Report_File'],'LAT:'  ).split()[0:4]
      opus_lon = get_file_var( rk['OPUS_Report_File'],'W LON:'  ).split()[0:5]
      opus_elev = get_file_var(rk['OPUS_Report_File'],'EL HGT:'  ).split()[2]
      opus_elev = float(opus_elev.split('(')[0])
      rk['Base_Position_Type']  = 'llh'
      rk['Base_Latitude_Y']  = opus_lat
      rk['Base_Longitude_X'] = opus_lon
      rk['Base_Elevation_Z'] = opus_elev
      k_settings['ant2-pos1']    = rk['Base_Latitude_Y']
      k_settings['ant2-pos2']    = rk['Base_Longitude_X']
      k_settings['ant2-pos3']    = rk['Base_Elevation_Z']
      k_settings['ant2-postype'] = rk['Base_Position_Type']
      print('Using the NAD83 Ellipsodial Coordinates from your OPUS Report File for your base station.')
    else:
      print('No OPUS Report file found.')
      rk['Base_data_ready_for_use'] = False

  if rk['base_files_exist']:
    strx = 'not set'
    if rk['Base_Coordinates_Source'] != 'RINEX Header':
      rk['Base_Latitude_Y'] = dms2dd(rk['Base_Latitude_Y'])
      rk['Base_Longitude_X'] = dms2dd( rk['Base_Longitude_X'])
      rk['Base_Elevation_Z'] = dms2dd( rk['Base_Elevation_Z'])
      rk['Base_Position_Type'] = 'llh'
      if rk['Base_Latitude_Y']  < -90.0 or rk['Base_Latitude_Y'] > 90.0:
        print(rk['Base_Latitude_Y'], 'Base_Latitude_X is invalid.')
      elif rk['Base_Longitude_X']  < -360.0 or rk['Base_Longitude_X'] > 360.0:
        print(rk['Base_Longitude_X'], 'Base_Longitude_X is invalid.')
      elif rk['Base_Elevation_Z'] < -900:
        print(rk['Base_Elevation_Z'], 'Base_Elevation_Z is invalid.')
      else:
        strx = "Base Station Coordinates: {:s}  Latitude: {:6.9F}  Longititude: {:6.9F} Elevation: {:6.3F} (m)".\
        format( rk['Base_Position_Type'], 
                rk['Base_Latitude_Y'], 
                rk['Base_Longitude_X'],
                rk['Base_Elevation_Z'] )
        if rk['Base_data_ready_for_use'] == True:
          k_settings['ant2-pos1']    = rk['Base_Latitude_Y']
          k_settings['ant2-pos2']    = rk['Base_Longitude_X']
          k_settings['ant2-pos3']    = rk['Base_Elevation_Z']
          k_settings['ant2-postype'] = rk['Base_Position_Type']
          print(strx )
    else:
      print('Using coordinates in the RINEX Header.')
      k_settings['ant2-postype'] = 'rinexhead'
  else:
    print('Please correct and retry.')
    rk['Base_data_ready_for_use'] = False 
  if rk['Base_data_ready_for_use']:
    display(HTML("<h2 style=""color:green""><b>Operation Completed ok.  Base ready for processing.</b></h2><hr>"))  
  else:
    display(HTML("<h2 style=""color:red""><b> Please Correct and retry</b></h2>"))
  return rk['Base_data_ready_for_use']


rk = {
    'Ephemeris_Type'          : '', 
    'Satellite_Systems'       : '',
    'Base_RINEX'              : '',
    'Base_RINEX_Nav'          : '',
    'Base_RINEX_Gnav'         : '',
    'Base_GPS_SP3'            : '',
    'Base_GLO_SP3'            : '',
    'Base_data_ready_for_use' : False,
    'Base_Coordinates_Source' : '',
    'Base_Longitude_X'        : '',
    'Base_Latitude_Y'         : '',
    'Base_Elevation_Z'        : '',
    'Base_Antenna'            : '',
    'OPUS_Report_File'        : ''
      }


# B. RTKlib Convert raw to RINEX

In [ ]:
#@title B1.0 GUI: Convert a CWW-PPK raw GPS data file to RINEX for processing. {form-width: "35%"}
#@markdown The first thing that must be done is converting your raw GNSS receiver data to RINEX.
#@markdown Use this cell to convert CWW-PPK raw data files to RINEX.  You need this for
#@markdown the UAS PPK data.  If you are using a second CWW-PPK in "Base Station" mode, you can
#@markdown it for that also.  If you are using another survey grade receiver, you will probably
#@markdown need to use tools specific to that receiver to convert it to RINEX.  RINEX is the required
#@markdown data format for use with this software package.
if __name__ == '__main__':
  Raw_File_Name = "/content/Gdrive/Missions/2022-0112-Nardi-Md/2022-0112-Nardi-Md_GNSS/aircraft/raw/GP165812.RAW" #@param {type:"string"}
  Data_Source   = "N7251F" #@param ["Solo", "M600", "N7251F", "CCX", "TFR"] {allow-input: true}
  Marker_Name   = "N7251F" #@param {type:"string"}
  User_Comment  = "Aircraft N7251F 2022-0112 Nardi.  Md marsh" #@param {type:"string"}

  rtkconvert(Raw_File_Name, Data_Source, Marker_Name, User_Comment)


# C.  RTKlib.  Configure Base and General Settings
Parameters

In [ ]:
#@title C0.1 **(Required data)** GUI: Load Base Station RINEX Files and settings to use. {form-width: "35%"}
import os
from   IPython.core.display import HTML, display



if __name__ == '__main__':
  Ephemeris_Type = "Ultra Rapid"         #@param ['Precise', 'Rapid', 'Ultra Rapid', 'Broadcast' ]
  Satellite_Systems= 'GPS'  #@param ['GPS', 'GPS_SBAS', 'GPS+GLONASS' ]
  Base_RINEX = "/content/Gdrive/MyDrive/Missions/2022-1028-NC/2022-1028-NC_GNSS/bases/CORS/ncbe/ncbe301-asof-2022-1028-223742.22o"           #@param {type:"string"}
  Base_RINEX_Nav = "/content/Gdrive/MyDrive/Missions/2022-1028-NC/2022-1028-NC_GNSS/bases/CORS/nav/brdc3010.22n"           #@param {type:"string"}
  Base_RINEX_Gnav = "/content/Gdrive/MyDrive/Missions/2022-1028-NC/2022-1028-NC_GNSS/bases/CORS/nav/brdc3010.22g"           #@param {type:"string"}
  Base_GPS_SP3= "/content/Gdrive/MyDrive/Missions/2022-1028-NC/2022-1028-NC_GNSS/bases/CORS/sp3/igu22335_18.sp3"           #@param {type:"string"}          
  Base_GLO_SP3= ""           #@param {type:"string"}
  Base_data_ready_for_use= True;
  rk['Ephemeris_Type'   ] = Ephemeris_Type
  rk['Satellite_Systems'] = Satellite_Systems
  rk['Base_RINEX'       ] = Base_RINEX
  rk['Base_RINEX_Nav'   ] = Base_RINEX_Nav
  rk['Base_RINEX_Gnav'  ] = Base_RINEX_Gnav
  rk['Base_GPS_SP3'     ] = Base_GPS_SP3
  rk['Base_GLO_SP3'     ] = Base_GLO_SP3
  rk['Base_data_ready_for_use'] = Base_data_ready_for_use


  #@markdown ---
  #@markdown Select 'RINEX Header' if you are using [CORS](https://www.ngs.noaa.gov/CORS_Map/) data as your base station.
  #@markdown Select 'Lat Lon Elevation' or 'X Y Z' if you are using your own local base station. 
  #@markdown 'X Y Z' coordinates are 'Earth Centered Earth Fixed' (ECEF).
  Base_Coordinates_Source = 'RINEX Header' #@param ['RINEX Header', 'Lat Lon Elevation', 'X Y Z', 'OPUS Report' ]
  Base_Longitude_X        = 'W LON:  105 11 34.33567'   #@param {type:"string"}
  Base_Latitude_Y         = 'LAT:   39 43 17.49313' #@param {type:"string"}
  Base_Elevation_Z        = '1819.826'  #@param {type:"string"} 

  rk['Base_Coordinates_Source'] = Base_Coordinates_Source
  rk['Base_Longitude_X'       ] = Base_Longitude_X 
  rk['Base_Latitude_Y'        ] = Base_Latitude_Y
  rk['Base_Elevation_Z'       ] = Base_Elevation_Z

  #@markdown Specify where to get the precise coordinates from.  
  #@markdown Select RINEX if you are using [CORS](https://www.ngs.noaa.gov/CORS_Map/)
  #@markdown data for your base station.  If you are using your own local base station, enter
  #@markdown the antenna identifier for your antenna.
  Base_Antenna= "RINEX Header" #@param ["RINEX Header", "CNTAT330", "Use Phase Center"] {allow-input: true}
  rk['Base_Antenna'] = Base_Antenna
  OPUS_Report_File = "/content/Gdrive/Missions/2022-0112-Nardi-Md/2022-0112-Nardi-Md_GNSS/bases/wff/opus/2021-0111-OPUS-report.txt" #@param {type:"string"}
  rk['OPUS_Report_File'] = OPUS_Report_File
  for f in rk['Base_RINEX'].split(','):
    f = f.strip()
    rk['Base_data_ready_for_use'] = rtklib_general_config(rk)
    if rk['Base_data_ready_for_use'] == False:
      break


# D. *RTKlib*. Process.


In [ ]:
#@title D0.1 Generate an RTKlib GNSS Trajectory. { form-width: "35%" }
if __name__ == '__main__':
  Position_Mode  = "kinematic"       #@param ['kinematic', 'single', 'dgps', 'static', 'fixed', 'ppp-static']
  Solution_Type  = "combined"        #@param ['combined', 'forward', 'backward' ]
  Elevaton_Mask =  15#@param ["5", "10", "12", "15", "18", "20", "22", "25", "27", "30", "33", "35", "40"] {type:"raw", allow-input: true}
  Aircraft_RINEX = "/content/Gdrive/MyDrive/Missions/2022-1028-NC/2022-1028-NC_GNSS/aircraft/rinex/2022-1028-150446-N7251F-gp150427.obs"         #@param {type:"string"}
  Trajectory_Folder = "/content/Gdrive/MyDrive/Missions/2022-1028-NC/2022-1028-NC_GNSS/aircraft/trajectories" #@param {type:"string"}
  Generate_Output_Trajectory = "Yes" #@param ["Yes", "No"]
  rk['Position_Mode'] = Position_Mode
  rk['Solution_Type'] = Solution_Type
  rk['Elevaton_Mask'] = Elevaton_Mask
  rk['Aircraft_RINEX'] = Aircraft_RINEX
  rk['Trajectory_Folder'] = Trajectory_Folder
  rk['Generate_Output_Trajectory'] = Generate_Output_Trajectory
  mprocess(rk)



# Notes, References, & Change logs

## Changes, Revision and bug fix Log
* 2020-0618
  * reorganized the code and outline
  * Changed to automatically make all directories in Trajectory_Folder if it doesn't already exist.
  * Changed to use the 08 atx antenna files instead of the 05 versions.
  * diverted processing error output to a std named output file.
  * Fixed bug with 'Ephemeris_Type'


## Development
Operationally, this notebook clones inself into /content/rtklib/RTKlib.py and then imports itself so that the functions distributed through the cells can all be loaded at once and avaliable for use without having to run any associated cells.


In [ ]:
#@title Optional. Install some RTKlib_example_data { form-width: "35%" }
if __name__ == '__main__':
  ! rm -rf /content/RTKlib_example_data/
  ! git clone https://github.com/lidar532/RTKlib_example_data.git
  ! cd /content/RTKlib_example_data/; gunzip -r *
  ! cd /content/RTKlib_example_data/2018-0810-NC/N7251F; cat 15*a* > 155157.RAW; rm -rf 15*a*;
  ! cd /content/RTKlib_example_data/2018-0326-Solo/bases/STXWW/; cat *raw.a* >2018-0326-STXWW.raw
  ! cd /content/RTKlib_example_data/2018-0326-Solo/bases/STXWW/; rm *raw.a*
  print('Operation Completed')

In [ ]:
#@title Test. list the functions loaded via dir()
if __name__ == '__main__':
  print(dir())

## References


## Notes
